<style>
rendered_html {
  font-size:0.8em;
}
</style>
<center>
    <h1>Computing huge subspaces of polynomials:<br>Symmetries to the rescue!</h1>

<br>
  <large><a href="http://Nicolas.Thiery.name">Nicolas M. Thiéry</a></large><br>
  <a href="http://lri.fr">Laboratoire de Recherche en Informatique</a><br>
  <a href="http://www.u-psud.fr">Université Paris Sud</a><br><br>
  
     
  Algebra and Combinatorics at LACIM <br>
  CRM 50th anniversary<br>
  September 28rd of 2018<br><br>
  
  Slides and code: https://github.com/nthiery/harmonic-modules<br>

  <a href="https://mybinder.org/v2/gh/nthiery/harmonic-modules/master?filepath=talk.ipynb">Live slides</a> on <a href="https://mybinder.org">Binder</a><br>
  
  Shorthand: https://tinyurl.com/nthiery-harmonic/
</center>

## Abstract

In spring 2017, I visited François Bergeron and we worked on his
favorite objects: the spaces H(n,k) of diagonal harmonic polynomials
in k sets of n variables. Those spaces connect to many areas of
algebraic combinatorics, representation theory and beyond, and the
case H(n,2) became famous a decade ago with the n! conjecture and
its proof by Haiman.

To fuel his ongoing studies François needed to compute the structure
of H(5,6). This is a space of dimension 6.10^5 made of polynomials
in 30 variables of degree up to 15, each having thousands of terms.

In this talk, I'll explain how the calculation can now be completed
in 45 minutes with a dozen cores and ~15Go of memory. This exploits
a combination of strategies (symmetries, representation theory of
the symmetric and general linear group, ...), each of which reduces
the complexity in time and memory by one or two orders of magnitude.

There will be little prerequisites and it's my hope that some
strategies (and maybe the code!) could be used in other contexts.

LaTeX definitions
$\def\QQ{\mathbb{Q}}$
$\def\NN{\mathbb{N}}$
$\def\x{\mathbf{x}}$
$\def\div{\operatorname{div}}$
$\def\harm{\operatorname{Harm}}$

## Motivations

<center>
    <img src="figures/crm.jpg"/ width="65%">
    <img src="figures/uqam.jpg"/ width="65%">
</center>

## Joint work with François Bergeron
<center><img src="figures/Francois.jpg" height="50%"></center>

And now **Pauline Hubert**

## One of François's favorite objects
<img src="figures/book.jpg" style="float:right;"/><br>
The space $\harm(n,r)$ of **diagonal harmonic polynomials** in
$$\QQ\begin{bmatrix}
        x_{11}&\cdots&x_{1n}\\
        \vdots & & \vdots\\
        x_{r1}&\cdots&x_{rn}\\
     \end{bmatrix}$$
- $\harm(n,1)$ : dim=$n!$<br>
  graded character: Hall-Littlewood Polynomials<br>
  connections with invariant theory, ...
- $\harm(n,2)$ : dim = $(n+1)^{n-1}$<br>
  graded character: Macdonald polynomials<br>
  $n!$-conjecture of Garsia/Bergeron<br>
  proof by Mark Haiman using the Hilbert scheme of points
- $\harm(n,r)$ : dim = ???

## Project: pushing further the computer exploration

**Aim**: dimension and graded character of $\harm(6,5)$

Brute force linear algebra?
- polynomials in 30 variables of degree up to 15, with thousands of terms
- dimension: $\approx$ 3.7 million

In [4]:
d = 3.7 * 10^6
t = d^3 / 10.^9 * units.time.second
t.convert(units.time.year)

1606.19609335363*year

In [6]:
var('z');
f = ( 1/(1-z)^30 ).series(z,16); f # the graded dimension of the polynomial ring Q[x_ij]

1 + 30*z + 465*z^2 + 4960*z^3 + 40920*z^4 + 278256*z^5 + 1623160*z^6 + 8347680*z^7 + 38608020*z^8 + 163011640*z^9 + 635745396*z^10 + 2311801440*z^11 + 7898654920*z^12 + 25518731280*z^13 + 78378960360*z^14 + 229911617056*z^15 + Order(z^16)

In [7]:
D = sum(f.coefficients(sparse=False))
t = d^1.8 * D / 10.^9 * units.time.second
t.convert(units.time.year)

7.27124321326576e6*year

### Intractable?

After a couple weeks of hard work, let's see what we get with 15 Gb and 10 cores:
- minutes? hours? days? years?
- dimension: ??

TODO:
- launch the calculation remotely instead of giving away the result now
- Configure partition output to be compact list, including in symmetric functions

In [4]:
%run old_code/code.py
%run -i talk.py
%display unicode_art

Compiling ./old_code/code1.pyx...


In [6]:
harmonic_bicharacter(3)

1 ⊗ s  + s  ⊗ s   + s   ⊗ s    + s  ⊗ s   + s  ⊗ s
     3    1    21    11    111    2    21    3    111

## Was it worth the trouble?

**Stability property**: $\harm(6,5)$ contains all the information about $\harm(6,r)$ for all $r$

«Coté mathématiques, grâce à tes calculs et à mon apprentissage de Sage, j’ai exploré une mine d’or de liens entre tout ce qu’on fait depuis le début sur les opérateurs de Macdonald, jusqu’à aujourd’hui; algèbre de Hall elliptique, Conjecture Delta, théorie des noeuds et  entrelacs du tore, etc. Je pense que cela sera ce que j’ai fait de mieux dans ma carrière. C’est magnifique et cela simplifie la compréhension de tout, permet de prouver plein de liens nouveaux, prédit comment généraliser, et de plus j’ai des modules qui expliquent cela.»

François

## Plan
### Motivations
### Warming up: harmonic polynomials
- Definition
- Exploiting the grading

### Diagonal harmonic polynomials
- Definition
- Exploiting the multigrading
- Exploiting symmetries: action of $S_r$
- Exploiting symmetries: action of $gl_r$?
- Exploiting symmetries: action of $S_n$
- Exploiting antisymmetries: action of $S_n$

## Harmonic polynomials
$\QQ[X]$: the ring of polynomials $f$ in the $n$ variables $X:=(x_1,\ldots,x_n)$

Partial derivatives: $\partial_i := \partial_{x_i} := \frac{\partial}{\partial x_i}$

**Definition:** $f\in\QQ[X]$ **harmonic** if
$$ \partial_1^k\,f+ \cdots + \partial_n^k\,f = 0, \qquad \forall k>0$$

In particular: $\quad \div f = 0, \quad  \nabla f = 0, \quad ...$

### Examples:

$1$, $x_1-x_2$ and linear combinations are harmonic

### The subspace $\harm(n)$  of harmonic polynomials

- the joint kernel of the differential operators $D_k := \partial_1^k + \cdots + \partial_n^k$

- a realization of the coinvariants of the symmetric group: $\QQ[X]\, /\, \langle Sym(X)^+ \rangle$

- the graded regular representation of $S_n$

### Example
$$\harm(2) = \langle 1, x_1-x_2 \rangle_\QQ$$

### Example on computer

In [11]:
%display unicode_art
R = QQ['x1,x2,x3']
x1,x2,x3 = X = R.gens()
p1 = lambda p: p.derivative(x1)                         # ∂₁
p2 = lambda p: p.derivative(x2)                         # ∂₂
p3 = lambda p: p.derivative(x3)                         # ∂₃
D1 = lambda p: sum( p.derivative(v) for v in X)         # ∂₁  + ∂₂  + ∂₃
D2 = lambda p: sum( p.derivative([v,v]) for v in X)     # ∂₁² + ∂₂² + ∂₃²
D3 = lambda p: sum( p.derivative([v,v,v]) for v in X)   # ∂₁³ + ∂₂³ + ∂₃³

In [13]:
D1(x1-x2)

0

In [14]:
D1(x1-x3), D1(x2-x3)

( 0, 0 )

In [15]:
D1(x1^2-x2^2)

2*x1 - 2*x2

In [16]:
Delta = ( (x1-x2) * (x1-x3) * (x2-x3) )
D1(Delta), D2(Delta), D3(Delta)

( 0, 0, 0 )

In [17]:
f = p1 ( Delta )
D1(f), D2(f), D3(f)

( 0, 0, 0 )

### What have we learned?
- $\Delta:=$ $\prod_{i<j}(x_i-x_j)$ is harmonic
- $f$ harmonic $\Longrightarrow$ $\partial_i f$ harmonic

**Theorem:**  $\harm(n) = \langle \Delta \rangle_{\partial_1,\ldots,\partial_n}$

#### On computer

In [19]:
%runfile code.py
H = Subspace([Delta], [p1,p2,p3])
H.dimension()

6

In [20]:
H.basis()[0]

( -x1^2*x2 + x1*x2^2 + x1^2*x3 - x2^2*x3 - x1*x3^2 + x2*x3^2,

 -x1*x2 + 1/2*x2^2 + x1*x3 - 1/2*x3^2, 1/2*x1^2 - x1*x2 + x2*x3 - 1/2*x3^2,

 x1 - x3, x2 - x3, 1 )

In [21]:
[ factor(v) for v in H.basis()[0]]

[ (x2 - x3) * (-x1 + x2) * (x1 - x3), (1/2) * (x2 - x3) * (-2*x1 + x2 + x3),

 (-1/2) * (-x1 + 2*x2 - x3) * (x1 - x3), x1 - x3, x2 - x3, 1 ]

### Algorithm: straightforward linear algebra in the monomial basis
`Subspace( polynomials, operators )`:
- maintain a list $L$ of all the relevant monomials
- represent the polynomials as finite vectors with columns indexed by $L$
- build a matrix and maintain it in row echelon form
- insert new vectors until the subspace is stable under the operators

**Tricky part**: make that work with most vector spaces in Sage

In [22]:
H.matrix()

⎛   1   -1    1    1   -1   -1    0    0    0    0    0    0    0    0    0    0⎞
⎜   0    0    0    0    0    0    1    0    0  1/2    0    0    0    0 -1/2   -1⎟
⎜   0    0    0    0    0    0    0    1  1/2    0    0    0    0    0 -1/2   -1⎟
⎜   0    0    0    0    0    0    0    0    0    0    1    0    0   -1    0    0⎟
⎜   0    0    0    0    0    0    0    0    0    0    0    1    0   -1    0    0⎟
⎝   0    0    0    0    0    0    0    0    0    0    0    0    1    0    0    0⎠

**Note**: The matrix is block diagonal: harmonic polynomials of degree $0, 1, 2, 3$

### Strategy: exploiting the grading

In [25]:
def add_degrees(d1, d2):
    d = d1 + d2
    if d < 0: raise ValueError("Negative degree")
    return d
F = Subspace(generators  = { 3: [Delta]},
             operators   = {-1: [p1,p2,p3]},
             add_degrees = add_degrees)

In [26]:
F.dimension()

6

In [27]:
F.hilbert_polynomial()

q^3 + 2*q^2 + 2*q + 1

**Algorithm**:
- construct the graph of the homogeneous components and operators between them
- maintain one matrix per homogeneous component and propagate polynomials

TODO: Draw the graph

**Potential gain** for an $n\times n$ matrix $M$ with $k$ blocks of same size:<br>
Complexity of linear algebra: $k\left(\frac n k\right)^3 = \frac 1{k^2} n^3$

So far, nothing very interesting, since $\harm(n,1)$ is very well known.
Time to move on to diagonal harmonic polynomials

## Diagonal harmonic polynomials
Polynomial ring in $r$ rows $X_1,\ldots,X_r=X,Y,\ldots,Z$ of $n$ variables:
$$\QQ\begin{bmatrix}
        x_1&\cdots&x_n\\
        \vdots & & \vdots\\
        z_1&\cdots&z_n\\
    \end{bmatrix}$$

Differential operators $(D_\alpha)_{\alpha\in \NN^r}$:
$$D_{(3,0,4)} := \partial_{x_1}^3\partial_{z_1}^4 + \dots + \partial_{x_n}^3\partial_{z_n}^4$$

**Diagonal harmonic polynomials** $\harm(n,r)$: joint kernel of the $(D_\alpha)_\alpha$

How to compute them?

## Computing diagonal harmonic polynomials

**Remark**: $\harm(n,r)$ contains $\harm(n,1)$

**Polarization operators:**<br>
- In two rows of variables:

$$P_{(-k,1)} := \partial_{x_1}^k y_1 +\cdots+\partial_{x_n}^k y_n$$

- Between any two rows $X_i$ and $X_j$ of variables:

$$P_{(0\ldots0,-k,0\ldots0,1,0\ldots0)} := \cdots$$ 

**Fact:**
$\harm(n,r)$ is generated by $\harm(n,1)$ and the polarization operators

## Multigrading
**Multidegree**: degree in each row $X_i$ of variables

**Example**: $x_1^2x_2 z_2^4$ is of multidegree $(3,0,4)=x^3z^4$

**Remark**:
- The $D_\alpha$ preserve the grading: $\harm(n,r)$ is multigraded
- The polarization operators preserve the multigrading
- So we can split the linear algebra by homogeneous components

TODO: Drawing of the graph
TODO: do the calculation directly with Subspace and polarization operators

## Example: computing the **Hilbert polynomial** of Harm(3,3):

<center><img src="figures/harmonic_graph-3-3-P.svg" width="60%"></center>

## Symmetries

## The Hilbert polynomial of Harm(3,3)

In [26]:
n = 3; r = 3
H = sum( harmonic_character(mu) * StandardTableaux(mu).cardinality()
         for mu in Partitions(n))
H.expand(r, 'xyz')

x^3 + x^2*y + x*y^2 + y^3 + x^2*z + x*y*z + y^2*z + x*z^2 + y*z^2 + z^3 + 2*x^2 + 3*x*y + 2*y^2 + 3*x*z + 3*y*z + 2*z^2 + 2*x + 2*y + 2*z + 1

**Note**: This is a symmetric polynomial. How comes?

**Remark**
- $\harm(n,r)$ stable under permuting rows of variables
- action of the symmmetric group $S_r$

TODO: Could show the matrices; or later, when illustrating antisymmetries

### Exploiting the action of $S_r$ on rows

- Restrict the computations to decreasing multidegrees:

<center><img src="figures/harmonic_graph-3-3-m.svg" width="40%"></center>

- Requires a bit of care: reordering after polarization

### Exploiting the action of $S_r$ on rows: gain

In [34]:
H.expand(r, 'xyz')

x^3 + x^2*y + x*y^2 + y^3 + x^2*z + x*y*z + y^2*z + x*z^2 + y*z^2 + z^3 + 2*x^2 + 3*x*y + 2*y^2 + 3*x*z + 3*y*z + 2*z^2 + 2*x + 2*y + 2*z + 1

In [35]:
sum(_.coefficients())

32

In [36]:
m(H)

1 + 2*m  + 3*m   + m    + 2*m  + m   + m
       1      11    111      2    21    3

In [37]:
sum(_.coefficients())

11

**In general**: an order of magnitude

### Back to the Hilbert polynomial of Harm(3,3)

In [42]:
m(H)

1 + 2*m  + 3*m   + m    + 2*m  + m   + m
       1      11    111      2    21    3

In [43]:
s(H)

1 + 2*s  + s   + 2*s  + s
       1    11      2    3

**Note**: it's Schur positive! How comes?

### The action of $GL(r)$ on rows

- $\harm(n,r)$ is stable under substituting linear combinations of rows
- Action of the general linear group $GL(r)$

<center><img src="figures/harmonic_graph-3-3-P.svg" width="60%"></center>

- The multi homogeneous components are the **weight spaces**
- The polarization operators $P_{(-1,1)}$: action on rows of the **lie algebra** $gl(r)$
- The Hilbert polynomial is the **$GL(r)$-character** of $\harm(n,r)$:  $1+2s_1 + 2s_2 +s_3 + s_{11}$

### Exploiting the action of $gl(r)$ on rows?
Hope: Restrict the computation to **highest weight spaces**:
<center><img src="figures/harmonic_graph-3-3-s.svg" width="30%"></center>

**Potential gain**:
- One or two orders of magnitude
- Finish computing $\harm(7,6)$
- François needs it! Presumably Vincent, Nantel, ... as well

### The action of $S_n$ on columns

Remember: $\Delta:=(x_1-x_2)(x_1-x_3)(x_2-x_3)$ is harmonic because of antisymmetries

- $\harm(n,r)$ is stable under permuting columns of variables

- Multigraded action of the symmetric group $S_n$ or column

- Polarization operators: $S_n$-morphisms

<center><img src="figures/harmonic_graph-3-3-P-s.svg" width="60%"></center>

### Exploiting the action of $S_n$ on columns: algorithm

**Step 1**: Choose a simple representation of $S_n$: $\lambda$

**Step 2**: Apply Young's idempotent $e_\lambda$ to harmonic polynomials

Optimization:
- Compute the **higher Specht** polynomials for $\lambda$
- Project onto harmonic polynomials     

**Step 3**: Apply polarization as usual

### Exploiting the action of $S_n$ on columns: gain?

In [39]:
n = 4; r = 3
sum(harmonic_character(mu) * StandardTableaux(mu).cardinality()
    for mu in Partitions(n))


1 + 3*s  + 3*s   + s    + 5*s  + 5*s   + 6*s  + 4*s   + 5*s  + s   + 3*s  + s
       1      11    111      2      21      3      31      4    41      5    6

In [40]:
{mu: harmonic_character(mu) for mu in Partitions(n) } 

⎰ 31:s  + s  + s , 1111:s    + s   + s   + s , 22:s  + s   + s , 211:s   + s  
⎱     1    2    3        111    31    41    6      2    21    4       11    21

 + s  + s   + s  + s , 4:1 ⎱
    3    31    4    5      ⎰

- Finer information: compute directly the $GL(r)-S_n$ bicharacter
- Smaller spaces: an order of magnitude
- Parallelism
- Partial computation: only two partitions missing for $n=7$

### The action of $S_n$ on columns: antisymmetries

In [45]:
n = 3; r = 3
H = DiagonalPolynomialRing(QQ,n,r)
HDelta = H.harmonic_space_by_shape([1]*n)
HDelta.finalize()
{mu:b._matrix for mu,b in HDelta._bases.iteritems()}

{ ( 0, 1, 1 ):( 1 -1 -1  1  1 -1),

 ( 2, 1, 0 ):(   1  1/2    1  1/2    1   -1 -1/2  1/2 -1/2 -1/2   -1   -1),

 ( 0, 0, 2 ):(), ( 0, 2, 0 ):(),

 ( 0, 1, 2 ):(   1    1  1/2 -1/2 -1/2   -1   -1    1  1/2  1/2   -1 -1/2),

 ( 1, 1, 0 ):( 1  1 -1 -1  1 -1),

 ( 1, 0, 2 ):(   1   -1  1/2    1  1/2   -1  1/2 -1/2    1 -1/2   -1 -1/2),

 ( 1, 0, 1 ):( 1 -1 -1  1  1 -1),

 ( 1, 1, 1 ):( 1 -1  1 -1  1 -1  1  1 -1 -1  1  1 -1 -1 -1  1 -1  1),

 ( 0, 3, 0 ):( 1 -1 -1 -1  1  1), ( 3, 0, 0 ):( 1 -1 -1 -1  1  1),

 ( 2, 0, 1 ):(   1  1/2    1 -1/2  1/2   -1   -1  1/2 -1/2    1   -1 -1/2),

 ( 0, 2, 1 ):(   1    1  1/2 -1/2 -1/2   -1   -1 -1/2   -1  1/2    1  1/2),

 ( 1, 2, 0 ):(   1 -1/2   -1  1/2  1/2   -1   -1 -1/2    1  1/2    1 -1/2),

 ( 0, 0, 3 ):( 1 -1 -1 -1  1  1) }

Lots of redundancy!!!

- $\Delta$ is antisymmetric:

In [42]:
Delta

x1^2*x2 - x1*x2^2 - x1^2*x3 + x2^2*x3 + x1*x3^2 - x2*x3^2

- Polarisation preserves antisymmetries

### Exploiting antisymmetries


#### Algorithm
For $\lambda = (1,\dots,1)$, $\Delta$:
- Represent a single monomial per $S_n$-orbit (the **diagram space** of François)
- Straighten after each application of polarization

Generalization to any partition:
- the Young idempotents creates partial antisymmetries by construction
- exploit them!
- straightening implemented in Cython

#### Gain
- A factor of up to $n!$

## Combining everything together?

# Time to wrapup
- We all know that symmetries should help
- We saw a prototypical example:<br>
  - reduction of a computation from years to minutes
  - using group representations
  - using lie algebra representations?

- It helped that we were just doing linear algebra!
- Using the underlying ideal structure?<br>
  Gröbner bases + symmetries? (known to be hard)

- Designing the code is half of the difficulty:<br>
  tension between simplicity for a given purpose and versatility

- Now used and further extended by **Pauline Hubert**

- **Project**: getting the Subspace code into Sage

 - Ask **Adriano** and **François** for the beautiful mathematics behind!

### Thank you LACIM, and happy anniversary!